In [1]:
from smartsim import Experiment
from smartredis import Client
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch.optim as optim
from sklearn.metrics import mean_squared_error
import numpy as np
from matplotlib import pyplot as plt

import io

# Set up the execution of the foamSmartSimMapField application 
# as a SmartSim Experiment. 
exp = Experiment("foam-smartsim-svd", launcher="local")

db = exp.create_database(port=8000,       # database port
                         interface="lo")  # network interface to use
exp.start(db)

# Connect the python client to the smartredis database
client = Client(address=db.get_address()[0], cluster=False)

# Serial execution run settings for foamSmartSimDmd
run_settings_serial = exp.create_run_settings(exe="foamSmartSimSvd", 
                                              exe_args="-case cavity -fieldName p")

# MPI parallel run settings for foamSmartSimMapFields - run_command can be "slurm" on a cluster.
num_mpi_ranks = 4
run_settings_parallel = exp.create_run_settings(exe="foamSmartSimSvd", 
                                                exe_args="-case cavity -fieldName p -parallel", 
                                                run_command="mpirun", 
                                                run_args={"np": f"{num_mpi_ranks}"})

openfoam_svd_model = exp.create_model(name="foamSmartSimSvd", run_settings=run_settings_parallel)

try:
     torch.set_default_dtype(torch.float64)
    
     # Run foamSmartSimSvd and do not block.
     exp.start(openfoam_svd_model, block=False)

     # Perform SVD for each MPI rank in SmartRedis - this should be distributed.
     

except Exception as e:
    print("Caught an exception: ", str(e))
    
finally:
    exp.stop(db)

SmartRedis Library@13-23-40:WARNING: Environment variable SR_LOG_FILE is not set. Defaulting to stdout
SmartRedis Library@13-23-40:WARNING: Environment variable SR_LOG_LEVEL is not set. Defaulting to INFO
13:23:40 argo SmartSim[11205] INFO Stopping model orchestrator_0 with job name orchestrator_0-CWWTWKSJ48HT
